# <span style="color:cornflowerblue"> Gerald Jones, Kimon Swanson, Alex Hines</span>
# <span style="color:cornflowerblue"> Home Work 4, Problem 1: Balanced Transportion Problem</span>
# <span style="color:cornflowerblue"> ISE522 Spg 22</span>

## Notebook Links/Sections:
1. [Data Display section](#Data-Display): Display of Data for warehouses and customers
2. [Model Formulation](#Model-Formulation): Mathematical formulation of problem
3. [Method Definitions](#Method-Definitions): Python code for various tasks
4. [Gurobi Implementation](#implementation):  Definition and omptimization with python and Gurobi
5. [Solution Discussion](#solution): A discussion and explanation of the solution

## Problem Description:
> A company supplies goods to <b>three customers</b>, who <b>each demand 30 units</b>. The company has <b>two warehouses</b>. <b>Warehouse 1 has 40 units</b> on hand and <b>Warehouse 2 has 30</b>. The <b>costs of shipping 1 unit from warehouse to a given customer are shown in the table displayed below in the [Data Display section](#Data-Display) </b>. There is a <b>penalty for unmet demand</b> that is <b>specific to each customer</b> and these are also displayed in the [Data Display section](#Data-Display). The task is to formulate this blanced transportation problem to <b>minimize the sum of the shortage and shipping costs</b>.

## Notes and Observations
* total supply across all locataion is less than total demand
* confilcting objectives:
    * increasing supply increases cost
    * not supplying what is demanded increases cost
* means need just the right amounts supplied from each warehouse to customer to get optimal solution


## Assumptions:
* units are integer values i.e. the smallest value of units above zero is 1
* Customers can be supplied by multiple warehouses to meet demands

# <span style="color:orange"><center><b>Module imports and data loading</b></center></span>

In [1]:
from _GUROBI_TOOLS_.GUROBI_MODEL_BUILDING_TOOLS import *
from _NOTE_BOOK_UTILS import *

notebook_title = "_HW4_Problem1.ipynb"

try:
    data_file1 = "WarehouseData.xlsx"
    data_file2 = "CustomerPenaltyData.xlsx"
    warehouse_df = pd.read_excel(data_file1)
    customer_df = pd.read_excel(data_file2)
except Exception as ex:
    print("error loading file")
    print("Exception: {}".format(ex))

<IPython.core.display.Javascript object>

# <a id=Data-Display><span style="color:Green"><center> Data Display</center></span></a>

In [2]:
print("\t\t\t\tWarehouse Data")
display(warehouse_df)
print("\t\t\t\tCustomer Data")
display(customer_df)

				Warehouse Data


,Warehouse,Customer-1,Customer-2,Customer-3
0,1,15,35,25
1,2,10,50,40


				Customer Data


,Customer,Penalty
0,1,90
1,2,80
2,3,110


# <a id=Model-Formulation><center> <span style="color:blue"> Model Formulation</span> </center></a>
### Model Formulation Links/Sections:
* [Paremeters and Sets](#Parameters-and-Sets)
* [Variables](#Variables)
* [Equations and Constraints](#Equations-and-Constraints)
* [Objective](#Objective)

## <a id=Parameters-and-Sets><span style="color:DarkBlue">Parameters and Sets:</span></a>

### $\textbf{W}  \quad \quad \quad \text{set of warehouses, } w \in \textbf{W}$ 
### $\textbf{C} \quad \quad \quad \text{set of customers, } c \in \textbf{C}$ 
### $D_{c} \quad \quad \quad \text{ demand for customer } c$
### $P_{c} \quad \quad \quad \text{ unmet penalty for customer } c$
### $H_{w} \quad \quad \quad \text{amount on hand for warehouse } w$
### $S_{w,c} \quad \quad \text{shipping costs for warehouse } w \text{ to customer } c$

## <a id=Variables><span style="color:DarkBlue">Variables:</span></a>

### $X_{w,c} \quad  \text{     amount from warehouse w supplied to customer } c$ 
### $Y_{c} \quad \quad   \text{    unmet demand for customer } c$
### $M \quad  \text{     total supply cost for warehouse} $ 
### $N \quad  \text{     total unmet demand cost}$ 

## <a id=Equations-and-Constraints><span style="color:DarkBlue">Equations and Constraints:</span></a>

>### <center><span style="font-size:30px;color:red"><b>Total units supplied by warehouse $w$ constraint</b></span></center>

# $$0 \leq \quad \sum_{c=1}^{|C|}X_{w,c} \quad  \leq H_{w}, \forall w$$

>### <center><span style="font-size:30px;color:red"><b>Total supply cost</b></span></center>

# $$M = \quad \sum_{w=1}^{|W|}\sum_{c=1}^{|C|} (X_{w,c} \cdot S_{w,c})$$

>### <center><span style="font-size:30px;color:red"><b>Total unmet demand for customer $c$</b></span></center>

# $$Y_{c} = \quad D_{c} - \sum_{w=1}^{|W|} (X_{w,c})  \quad, \forall c$$
# $$Y_{c} \geq 0, \forall c$$

>### <center><span style="font-size:30px;color:red"><b>Total unmet demand cost</b></span></center>

# $$N = \quad \sum_{c=1}^{|C|} (Y_{c} \cdot P_{c}) \quad, \forall c$$

## <a id=Objective><span style="color:green">Objective: Minimize total costs for shipping and unmet demand</span></a>

# $$\min(N + M)$$

# <a id=Method-definitions><center>Method Definitions</center></a>

In [3]:
# generate constraints for the amount a given warehouse
# can supply
def Xwc_supply_constraints(model, X, onHands, W, C):
    
    for w in range(W):
        expression = 0
        for c in range(C):
            expression += X[w, c]
        model.addConstr(expression <= onHands[w])
        # it has to supply something
        model.addConstr(expression >= 1)
    return

# set expression for total supply costs
def total_supply_cost(model, M, X, S, W, C):
    expression = 0
    for w in range(W):
        for c in range(C):
            expression += X[w, c] * S[w, c]
    model.addConstr(M == expression)
    model.addConstr(M >= 1)

    

# set expression for each customers unmet demand    
def set_unmet_demand(model, D, X, Y, C, W):
    # for each customer
    for c in range(C):
        expression = 0
        # sum up the contribution to its demand from each warehouse
        for w in range(W):
            expression += X[w, c] 
        # the current customers unmet demand is its demand minus what it was supplied
        # by the warehouses
        model.addConstr(Y[c] == D[c] - expression )
        
        # the unmet demand can at least be zero
        # this ensures that the sum of the supplied demand can not exceed the demand itself
        model.addConstr(Y[c] >= 1)
    return

# set expression for total unmet demand costs
def total_unmet_demand_cost(model, N, Y, D, P, C):
    expression = 0
    for c in range(C):
        expression += Y[c] * P[c]
    model.addConstr(N == expression)
    model.addConstr(N >= 0)
    return

# <a id=implementation><center>Gurobi Implementation and Solution</center></a>

In [4]:
try:
    # instantiate model object 
    m = gp.Model("G_MOD")
 
    
    
    #########################################################################################
    ################################## Parameters set up ####################################
    #########################################################################################
    W = 2                                                          # number of warehouses
    C = customer_df.shape[0]                                       # number of customers
    Pc = customer_df["Penalty"].tolist()                           # unmet demand penalties
    Hw = [40, 30]                                                  # on hand for each warehouse
    
    customer_labels = ["Customer-1", "Customer-2", "Customer-3"]
    ShippingCosts = warehouse_df.loc[:,customer_labels ].values    # shipping costs 
    print(ShippingCosts)
    
    Dc = [30, 30, 30]                                              # customer demands
    
    print("Parameters:\nW={}\nC={}\nPc={}\nHw={}\n".format(W, C, Pc, Hw))
    
    #########################################################################################
    ################################## Variables set up #####################################
    #########################################################################################
    Xwc = m.addVars(W, C, vtype=GRB.INTEGER, name="X", lb=0,)
    Yc = m.addVars(C, vtype=GRB.INTEGER, name="Y", lb=0,)
    M = m.addVar(1, vtype=GRB.CONTINUOUS, name="M",)
    N = m.addVar(1, vtype=GRB.CONTINUOUS, name="N")
    
    
    #########################################################################################
    ################################## Objective set up #####################################
    #########################################################################################    
    m.setObjective(N+M, GRB.MINIMIZE)
    
    
    #########################################################################################
    ################################## Constraint set up ####################################
    #########################################################################################
    Xwc_supply_constraints(m, Xwc, Hw, W, C)
    
    
    # set the total shipping cost
    total_supply_cost(m, M, Xwc, ShippingCosts, W, C)
    
    # set the unmet demand expression
    set_unmet_demand(m, Dc, Xwc, Yc, C, W)
    
    # set total unmet demand cost expression 
    total_unmet_demand_cost(m, N, Yc, Dc, Pc, C)
    
    #########################################################################################
    ################################## SOLVE:OPTIMIZE #######################################
    #########################################################################################    
    
    
    m.optimize()
    
    #########################################################################################
    ################################## Display Results ######################################
    #########################################################################################    
    displayDecisionVars(m, end_sentinel="2")
    
    print("\n-------------Does it make sense?----------------------")  
    print('Obj: {:.2f}'.format(m.ObjVal))
    
    
# catch some math errors
except gp.GurobiError as e:
    print('Error code ' + str(e.errno) + ': ' + str(e))

except AttributeError:
    print('Encountered an attribute error')

Restricted license - for non-production use only - expires 2023-10-25
[[15 35 25]
 [10 50 40]]
Parameters:
W=2
C=3
Pc=[90, 80, 110]
Hw=[40, 30]

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 14 rows, 11 columns and 37 nonzeros
Model fingerprint: 0xbf04315a
Variable types: 2 continuous, 9 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+01]
Presolve removed 7 rows and 5 columns
Presolve time: 0.00s
Presolved: 7 rows, 6 columns, 18 nonzeros
Variable types: 0 continuous, 6 integer (0 binary)
Found heuristic solution: objective 3730.0000000

Root relaxation: objective 3.090000e+03, 5 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap

# <a id=solution><span style="color:crimson"><center>Solution Discussion</center></a>

> ### The solution....
> The problem requires that a decision be made about how much each warehouse will supply to which customer. Looking at the unmet demand penalties the general idea should be to supply the customers with the higher penalties more than those with lower penalties to minimize unmet demand cost. From the supply costs from each warehouse to each customer the warehouse 1 has lower supply costs overall compared to warehouse 2 for any given customer.  Since warehouse 1 has more supply, I would expect that it supplies most of the demand with warehouse 2 picking up the slack. For both warehouses the magnitude of demand in ascending order is customers 1, 2, and 3. Since customer 1 has a lower cost for warehouse 2 I would try to send most of the supply for that customer from warehouse 2. This would mean that warehouse 1 would need to cover most of the demands for customers 2, and 3. Customers 1 and 3 have higher unmet demand penalties than customer 2 so I would allow for more unmet demand to this customer. I would supply what I had left over in warehouse 1 to customer 2 to met what demand I could.  I would supply as close to what customer 3 demanded as possible since it has the highest unmet demand cost. This ad hoc solution is like what is seen in the solution produced by Gurobi. 

> ### The optimal solution generated by the implemented model suggests to:
> * <b>Have Warehouse 1:</b>
>    * supplies most of <b>customers 3’s</b> demand <b>(28/30 units)</b> 
>    * as well as a portion of the demand for <b>customer 2 (12/30 units)</b> 
> * <b>Have Warehouse 2:</b>
>    * covers almost all the demand for <b>customer 1 (29/30 units)</b> 
>    * as well as <b>(1/30) units for customer 3</b>. 
    
> The leads to customer 1 supplied with 28 units, customer 2 with 12 units, and customer 3 with 29 units.</b> This configuration of the model leads to a <b>total supply cost of $\$1450$, and a total unmet demand cost of $\$1640$ for a total overall cost of $\$3090.00$</b>.

In [5]:
# save the notebook as a pdf
to_PDF(notebook_title)

filename: _HW4_Problem1.ipynb
